In [1]:
# imports

from keras.applications.xception import Xception, preprocess_input
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy
import json
from pandas.io.json import json_normalize
import string
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
import time, sys
from IPython.display import clear_output
from PIL import Image
import numpy as np

tqdm().pandas()

2022-04-18 18:16:28.261569: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-18 18:16:28.261590: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/tmp/ipykernel_7732/2053491181.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [2]:
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [3]:
# create train, validation, and test sets as dataframes

# training data
with open('data/annotations/train.json') as json_train:
    train_data = json.load(json_train)    
train_imgs_df = pd.DataFrame(train_data['images'])
train_captions_df = pd.DataFrame(train_data['annotations'])

#validation data
with open('data/annotations/val.json') as json_val:
    val_data = json.load(json_val)
val_imgs_df = pd.DataFrame(val_data['images'])
val_captions_df = pd.DataFrame(val_data['annotations'])

#test data
with open('data/annotations/test.json') as json_test:
    test_data = json.load(json_test)
test_imgs_df = pd.DataFrame(test_data['images'])

In [4]:
train_imgs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23431 entries, 0 to 23430
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   file_name      23431 non-null  object
 1   vizwiz_url     23431 non-null  object
 2   id             23431 non-null  int64 
 3   text_detected  23431 non-null  bool  
dtypes: bool(1), int64(1), object(2)
memory usage: 572.2+ KB


In [5]:
train_captions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117155 entries, 0 to 117154
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   caption        117155 non-null  object
 1   image_id       117155 non-null  int64 
 2   is_precanned   117155 non-null  bool  
 3   is_rejected    117155 non-null  bool  
 4   id             117155 non-null  int64 
 5   text_detected  117155 non-null  bool  
dtypes: bool(3), int64(2), object(1)
memory usage: 3.0+ MB


In [6]:
val_imgs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7750 entries, 0 to 7749
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   file_name      7750 non-null   object
 1   vizwiz_url     7750 non-null   object
 2   id             7750 non-null   int64 
 3   text_detected  7750 non-null   bool  
dtypes: bool(1), int64(1), object(2)
memory usage: 189.3+ KB


In [7]:
val_captions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38750 entries, 0 to 38749
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   caption        38750 non-null  object
 1   image_id       38750 non-null  int64 
 2   is_precanned   38750 non-null  bool  
 3   is_rejected    38750 non-null  bool  
 4   id             38750 non-null  int64 
 5   text_detected  38750 non-null  bool  
dtypes: bool(3), int64(2), object(1)
memory usage: 1021.9+ KB


In [8]:
test_imgs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   file_name      8000 non-null   object
 1   vizwiz_url     8000 non-null   object
 2   id             8000 non-null   int64 
 3   text_detected  8000 non-null   bool  
dtypes: bool(1), int64(1), object(2)
memory usage: 195.4+ KB


In [9]:
#train_captions_df.head(30)

In [10]:
len(train_imgs_df) + len(val_imgs_df) + len(test_imgs_df)

39181

In [11]:
len(train_captions_df) + len(val_captions_df)

155905

In [12]:
train_imgs_df.head()

,file_name,vizwiz_url,id,text_detected
0,VizWiz_train_00000000.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualiz...,0,True
1,VizWiz_train_00000001.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualiz...,1,True
2,VizWiz_train_00000002.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualiz...,2,True
3,VizWiz_train_00000003.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualiz...,3,True
4,VizWiz_train_00000004.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualiz...,4,True


In [13]:
train_captions_df.head()

,caption,image_id,is_precanned,is_rejected,id,text_detected
0,ITS IS A BASIL LEAVES CONTAINER ITS CONTAINS T...,0,False,False,0,True
1,A green and white plastic condiment bottle con...,0,False,False,1,True
2,Quality issues are too severe to recognize vis...,0,True,True,2,True
3,A bottle of spices in a plastic container layi...,0,False,False,3,True
4,some basil leaves in a container on a counter,0,False,False,4,True


In [14]:
# I need to create a dictionary where the keys are the image files and the captions are the values

def image_caption_connector(images,captions):
    
    descriptions = {}
    cap_idx = 0
    
    for img in range(len(images)):
        caps = []
        
        for cap in captions['caption'][cap_idx:cap_idx+5]:
            caps.append(cap)
            
            if len(caps) > 4:
                cap_idx += 5
                break
            
        current_img = images['file_name'][img]
        descriptions[current_img] = caps
            
    return descriptions

In [15]:
#image_caption_connector(train_imgs_df[:5], train_captions_df[:25])

In [16]:
img_cap_dict = image_caption_connector(train_imgs_df, train_captions_df)

In [17]:
val_cap_dict = image_caption_connector(val_imgs_df, val_captions_df)

In [18]:
#img_cap_dict

In [19]:
key_check = list(img_cap_dict)[19763]
val_check = list(img_cap_dict.values())[19763]

In [20]:
key_check

'VizWiz_train_00019763.jpg'

In [21]:
val_check

['Quality issues are too severe to recognize visual content.',
 'Wrist of a person wearing a silver watch holding something',
 'A human hand and arm, wearing a watch and  holding something blue just out of frame.',
 "A photo of someone's left hand wearing a silver watch holding an object.",
 'a hand wearing a metal watch carrying a hard to identify object']

# A Note

Not entirely sure why, but the local images folder on my pc has 23953 images in it, but the .json file appears to only be bringing the first 23430 into a dataframe. This represents ~2% of overall training data that I have no annotations for and thus can't use. Probably not a big deal but I should still note this, right?

In [22]:
# now to clean the text in a few basic ways

def text_cleaner(img_caps):
    table = str.maketrans('','',string.punctuation)
    
    for img,cap in img_caps.items():
        
        for i,caption in enumerate(cap):
            caption.replace("-"," ")
            desc = caption.split()
            
            #lowercase
            desc = [w.lower() for w in desc]
            
            #remove punctuation
            desc = [w.translate(table) for w in desc]
            
            #remove digits
            desc = [w for w in desc if(w.isalpha())]
            
            #convert back to string
            caption = ' '.join(desc)
            img_caps[img][i] = caption
    
    return img_caps

In [23]:
cleaned_caps = text_cleaner(img_cap_dict)

In [24]:
val_cleaned_cap = text_cleaner(val_cap_dict)

In [25]:
key_check = list(img_cap_dict)[19763]
val_check = list(img_cap_dict.values())[19763]

In [26]:
key_check

'VizWiz_train_00019763.jpg'

In [27]:
val_check

['quality issues are too severe to recognize visual content',
 'wrist of a person wearing a silver watch holding something',
 'a human hand and arm wearing a watch and holding something blue just out of frame',
 'a photo of someones left hand wearing a silver watch holding an object',
 'a hand wearing a metal watch carrying a hard to identify object']

In [28]:
# create a vocabulary of all the unique words

def vocabber(corpus):
    vocab = set()
    
    for k in corpus.keys():
        [vocab.update(c.split()) for c in corpus[k]]
    
    return vocab

In [29]:
vocabulary = vocabber(img_cap_dict)

In [30]:
val_vocab = vocabber(val_cap_dict)

In [31]:
len(val_vocab)

11007

In [32]:
# deconstruct the dictonary and organize it into one text file

def save_progress(captions, filename):
    lines = []
    
    for k, caps in captions.items():
        for cap in caps:
            lines.append(k + '\t' + cap)
    
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [45]:
save_progress(cleaned_caps, 'images_with_associated_captions.txt')

In [46]:
save_progress(val_cleaned_cap, 'val_imgs_w_caps.txt')

In [38]:
img_names = []

for img in train_imgs_df['file_name']:
    img_names.append(img)

In [39]:
len(img_names)

23431

In [40]:
with open("img_names.txt", "w") as imgs:
    imgs.write(str(img_names))

In [41]:
val_img_names = []

for img in val_imgs_df['file_name']:
    val_img_names.append(img)

In [42]:
with open("val_img_names.txt", "w") as imgs:
    imgs.write(str(val_img_names))

In [39]:
# extract features from VALIDATION mages using Xception transfer learning

def get_val_img_features(filepath):

    xfer_model = Xception(include_top=False, pooling='avg')
    features = {}

    for img in tqdm(os.listdir(filepath)):
        #print(type(img))  
        #img_name = subfolder + '/' + img
        #print(img)
        img_name = 'data/vizwiz data/val' + '/' + img
        image = Image.open(img_name.li.)
        image = image.resize((299,299))
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image -= 1.0

        feature = xfer_model.predict(image)
        features[img] = feature

    return features

In [40]:
val_img_path = 'data/vizwiz data/val'

In [45]:
val_features = get_val_img_features(val_img_path)

/tmp/ipykernel_14777/1444478676.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for img in tqdm(os.listdir(filepath)):


  0%|          | 0/7750 [00:00<?, ?it/s]

In [46]:
len(val_features)

7750

In [47]:
dump(val_features, open('val_features.p', 'wb'))

In [51]:
key_check = list(val_features)[0]
val_check = list(val_features.values())[0]

In [52]:
key_check

'VizWiz_val_00000453.jpg'

In [53]:
val_check

array([[0.10363426, 0.14685044, 0.00564226, ..., 0.        , 0.16573177,
        0.16373302]], dtype=float32)